# 使用 RNN 预测电力变压器油温 / Power Transformer Oil Temperature Prediction using RNN

## 简介 / Introduction

本 notebook 实现了一个循环神经网络（RNN）模型来预测电力变压器的油温（OT）。

This notebook implements a Recurrent Neural Network (RNN) model for predicting the oil temperature (OT) of power transformers.

**数据集 / Dataset**: ETDataset (电力变压器温度数据集 / Electricity Transformer Temperature)
- 来源 / Source: https://github.com/zhouhaoyi/ETDataset
- 出处 / From: AAAI 2021 最佳论文（Informer 模型）/ AAAI 2021 Best Paper (Informer model)

**特征 / Features**:
- HUFL: 高压有功负载 / High UseFul Load
- HULL: 高压无功负载 / High UseLess Load  
- MUFL: 中压有功负载 / Medium UseFul Load
- MULL: 中压无功负载 / Medium UseLess Load
- LUFL: 低压有功负载 / Low UseFul Load
- LULL: 低压无功负载 / Low UseLess Load

**目标变量 / Target**: OT (油温 / Oil Temperature)

**目标 / Goal**: 基于历史负载数据构建 RNN 模型预测油温，并展示：
- 时间序列数据预处理 / Time series data preprocessing
- 用于序列预测的 RNN 架构 / RNN architecture for sequence prediction
- 模型训练与评估 / Model training and evaluation
- 性能指标与可视化 / Performance metrics and visualization

## 步骤 1: 导入所需库 / Step 1: Import Required Libraries

我们将使用 / We'll use:
- pandas/numpy: 数据处理 / data manipulation
- sklearn: 预处理和评估指标 / preprocessing and metrics
- **PyTorch**: 构建 RNN 模型 / building the RNN model
- matplotlib: 可视化 / visualization

In [ ]:
# 数据处理 / Data manipulation
import pandas as pd
import numpy as np

# 预处理 / Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 深度学习 / Deep Learning - PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 可视化 / Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# 工具 / Utilities
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子以保证可复现性 / Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# 设备配置 / Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"PyTorch 版本 / PyTorch version: {torch.__version__}")
print(f"设备 / Device: {device}")
print(f"CUDA 可用 / CUDA available: {torch.cuda.is_available()}")

## 步骤 2: 加载和探索数据 / Step 2: Load and Explore Data

我们将加载变压器温度数据集并进行初步探索以了解：

We'll load the transformer temperature dataset and perform initial exploration to understand:
- 数据形状和结构 / Data shape and structure
- 缺失值 / Missing values
- 统计属性 / Statistical properties
- 时间序列特征 / Time series characteristics

In [ ]:
def load_data(filepath):
    """
    Load the ETT dataset
    加载 ETT 数据集
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file
    
    Returns:
    --------
    df : pd.DataFrame
        Loaded dataframe with date as index
    """
    df = pd.read_csv(filepath)
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    return df

# Load pre-split training and test data
# 加载预先切分的训练集和测试集
train_filepath = '../dataset/processed_data/train.csv'
test_filepath = '../dataset/processed_data/test.csv'

df_train_full = load_data(train_filepath)
df_test_full = load_data(test_filepath)

# For initial exploration and model training, we'll use the training data
# The test data will be used only for final evaluation
# 初步探索和模型训练使用训练数据，测试数据仅用于最终评估
df = df_train_full

print("Dataset loaded from pre-split files:")
print(f"  Training data: {train_filepath}")
print(f"  Test data: {test_filepath}")
print(f"\nTraining set shape: {df_train_full.shape}")
print(f"Test set shape: {df_test_full.shape}")
print(f"\nFirst few rows of training data:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nBasic Statistics:")
print(df.describe())
print("\nMissing Values:")
print(df.isnull().sum())

## 步骤 3: 数据可视化 / Step 3: Data Visualization

可视化时间序列以理解模式和关系

Visualize the time series to understand patterns and relationships

In [ ]:
# Plot all features over time
# 绘制所有特征随时间的变化
fig, axes = plt.subplots(4, 2, figsize=(15, 12))
fig.suptitle('Time Series of All Features / 所有特征的时间序列', fontsize=16)

for idx, col in enumerate(df.columns):
    ax = axes[idx // 2, idx % 2]
    ax.plot(df.index[:2000], df[col][:2000])  # Plot first 2000 points for clarity
    ax.set_title(col)
    ax.set_xlabel('Time / 时间')
    ax.set_ylabel('Value / 值')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation heatmap
# 相关性热力图
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.2f')
plt.title('Feature Correlation Heatmap / 特征相关性热力图')
plt.tight_layout()
plt.show()

## 步骤 4: 数据预处理 / Step 4: Data Preprocessing

### 4.1 特征和目标分离 / Feature and Target Separation

我们将特征（负载数据）与目标（OT - 油温）分离

We'll separate features (load data) from target (OT - Oil Temperature)

In [ ]:
def prepare_features_target(df):
    """
    Separate features and target variable
    分离特征和目标变量
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    
    Returns:
    --------
    X : np.ndarray
        Features (all columns except OT)
    y : np.ndarray
        Target (OT column)
    """
    # Features: all columns except OT
    # 特征：除 OT 外的所有列
    X = df.drop('OT', axis=1).values
    # Target: OT (Oil Temperature)
    # 目标：OT（油温）
    y = df['OT'].values
    
    return X, y

X, y = prepare_features_target(df)
print(f"Features shape / 特征形状: {X.shape}")
print(f"Target shape / 目标形状: {y.shape}")
print(f"\nFeature names / 特征名称: {df.drop('OT', axis=1).columns.tolist()}")

### 4.2 数据标准化 / Data Normalization

标准化特征和目标以提高 RNN 训练稳定性和收敛性

Normalize features and target to improve RNN training stability and convergence

In [ ]:
def normalize_data(X, y):
    """
    Normalize features and target using StandardScaler
    使用 StandardScaler 标准化特征和目标
    
    Parameters:
    -----------
    X : np.ndarray
        Features
    y : np.ndarray
        Target
    
    Returns:
    --------
    X_scaled : np.ndarray
        Normalized features
    y_scaled : np.ndarray
        Normalized target
    scaler_X : StandardScaler
        Fitted scaler for features
    scaler_y : StandardScaler
        Fitted scaler for target
    """
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
    
    return X_scaled, y_scaled, scaler_X, scaler_y

X_scaled, y_scaled, scaler_X, scaler_y = normalize_data(X, y)
print("Data normalized successfully! / 数据标准化成功！")
print(f"\nFeatures - Mean: {X_scaled.mean():.4f}, Std: {X_scaled.std():.4f}")
print(f"Target - Mean: {y_scaled.mean():.4f}, Std: {y_scaled.std():.4f}")

### 4.3 创建时间序列序列 / Create Time Series Sequences

将数据转换为 RNN 输入序列。我们使用滑动窗口方法：

Transform data into sequences for RNN input. We use a sliding window approach:
- 输入：过去 `seq_length` 个时间步 / Input: Past `seq_length` time steps
- 输出：下一个时间步的油温 / Output: Next time step's oil temperature

In [ ]:
def create_sequences(X, y, seq_length=24):
    """
    Create sequences for time series prediction
    创建时间序列预测的序列
    
    Parameters:
    -----------
    X : np.ndarray
        Features array
    y : np.ndarray
        Target array
    seq_length : int
        Number of time steps to look back (default: 24 = 6 hours with 15-min intervals)
        回溯的时间步数（默认：24 = 15分钟间隔下的6小时）
    
    Returns:
    --------
    X_seq : np.ndarray
        Sequences of features (samples, seq_length, n_features)
    y_seq : np.ndarray
        Target values for each sequence
    """
    X_seq = []
    y_seq = []
    
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    
    return np.array(X_seq), np.array(y_seq)

# Create sequences with lookback window of 24 time steps (6 hours)
# 创建回溯窗口为24个时间步（6小时）的序列
seq_length = 24
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

print(f"Sequence length (lookback window) / 序列长度（回溯窗口）: {seq_length}")
print(f"X_seq shape: {X_seq.shape} (samples, time_steps, features)")
print(f"y_seq shape: {y_seq.shape}")
print(f"\nExample / 示例:")
print(f"- Input / 输入: {seq_length} time steps of {X_seq.shape[2]} features")
print(f"- Output / 输出: 1 time step oil temperature prediction")

### 4.4 训练集/测试集分割 / Train/Test Split

将数据分割为训练集和测试集。对于时间序列，我们使用时间分割（非随机）。

Split data into training and testing sets. For time series, we use temporal split (not random).

In [ ]:
# Since data is already pre-split into train and test sets,
# we process them separately and create sequences for each
# 由于数据已经预先分割为训练集和测试集，我们分别处理它们并为每个创建序列

# Process training data / 处理训练数据
X_train_full, y_train_full = prepare_features_target(df_train_full)
X_train_scaled, y_train_scaled, scaler_X, scaler_y = normalize_data(X_train_full, y_train_full)
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, seq_length)

# Further split training data for validation (80% train, 20% validation)
# 进一步分割训练数据用于验证（80% 训练，20% 验证）
val_split_idx = int(len(X_train_seq) * 0.8)
X_train = X_train_seq[:val_split_idx]
y_train = y_train_seq[:val_split_idx]
X_val = X_train_seq[val_split_idx:]
y_val = y_train_seq[val_split_idx:]

# Process test data (using fitted scalers from training)
# 处理测试数据（使用从训练集拟合的缩放器）
X_test_full, y_test_full = prepare_features_target(df_test_full)
X_test_scaled = scaler_X.transform(X_test_full)
y_test_scaled = scaler_y.transform(y_test_full.reshape(-1, 1)).flatten()
X_test, y_test = create_sequences(X_test_scaled, y_test_scaled, seq_length)

print("Data prepared from pre-split files! / 从预分割文件准备数据完成！")
print(f"\nTraining set (for model fitting) / 训练集（用于模型拟合）:")
print(f"  X_train shape: {X_train.shape}")
print(f"  y_train shape: {y_train.shape}")
print(f"\nValidation set (from training data) / 验证集（来自训练数据）:")
print(f"  X_val shape: {X_val.shape}")
print(f"  y_val shape: {y_val.shape}")
print(f"\nTest set (held-out data) / 测试集（保留数据）:")
print(f"  X_test shape: {X_test.shape}")
print(f"  y_test shape: {y_test.shape}")
print(f"\nNote: Scalers fitted on training data and applied to test data")
print(f"注意：缩放器在训练数据上拟合并应用于测试数据")

## 步骤 5: 构建 RNN 模型 / Step 5: Build RNN Model

我们将构建一个简单的 RNN 模型，包含：

We'll build a Simple RNN model with:
- 输入层：特征序列 / Input layer: sequences of features
- RNN 层：捕获时间依赖关系 / RNN layers: to capture temporal dependencies
- Dropout 层：用于正则化 / Dropout layers: for regularization
- 全连接输出层：用于回归 / Dense output layer: for regression

In [ ]:
class SimpleRNNModel(nn.Module):
    """
    Simple RNN model for time series prediction
    用于时间序列预测的简单 RNN 模型
    
    Architecture / 架构:
    - RNN Layer 1: input_size -> hidden_size[0]
    - Dropout
    - RNN Layer 2: hidden_size[0] -> hidden_size[1]
    - Dropout
    - Fully Connected Layer 1: hidden_size[1] -> 16
    - ReLU activation
    - Dropout
    - Output Layer: 16 -> 1
    """
    
    def __init__(self, input_size, hidden_sizes=[64, 32], dropout=0.2):
        super(SimpleRNNModel, self).__init__()
        
        # First RNN layer
        self.rnn1 = nn.RNN(input_size, hidden_sizes[0], batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        
        # Second RNN layer
        self.rnn2 = nn.RNN(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.dropout2 = nn.Dropout(dropout)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_sizes[1], 16)
        self.relu = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout * 0.5)
        self.fc2 = nn.Linear(16, 1)
    
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        
        # RNN layer 1
        out, _ = self.rnn1(x)  # out shape: (batch_size, seq_length, hidden_sizes[0])
        out = self.dropout1(out)
        
        # RNN layer 2 (only use last output)
        out, _ = self.rnn2(out)  # out shape: (batch_size, seq_length, hidden_sizes[1])
        out = self.dropout2(out)
        out = out[:, -1, :]  # Take only last time step: (batch_size, hidden_sizes[1])
        
        # Fully connected layers
        out = self.fc1(out)  # (batch_size, 16)
        out = self.relu(out)
        out = self.dropout3(out)
        out = self.fc2(out)  # (batch_size, 1)
        
        return out.squeeze()  # (batch_size,)

# Model parameters / 模型参数
input_size = X_train.shape[2]  # Number of features / 特征数量
hidden_sizes = [64, 32]  # RNN hidden units / RNN 隐藏单元
dropout = 0.2  # Dropout rate / Dropout 率

# Initialize model / 初始化模型
model = SimpleRNNModel(input_size, hidden_sizes, dropout).to(device)

print("Model Architecture / 模型架构:")
print("=" * 60)
print(model)
print("=" * 60)

# Count parameters / 统计参数
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal Parameters / 总参数数: {total_params:,}")
print(f"Trainable Parameters / 可训练参数数: {trainable_params:,}")

## 步骤 6: 训练模型 / Step 6: Train the Model

使用以下方法训练 RNN：

Train the RNN with:
- Adam 优化器 / Adam optimizer
- MSE 损失函数 / MSE loss function
- 学习率调度器：提高收敛性 / Learning rate scheduler: improve convergence
- 早停：防止过拟合 / Early stopping: prevent overfitting

In [ ]:
# Convert numpy arrays to PyTorch tensors
# 将 numpy 数组转换为 PyTorch 张量
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create TensorDatasets
# 创建 TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
# 创建 DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Batch size / 批次大小: {batch_size}")
print(f"Number of training batches / 训练批次数: {len(train_loader)}")
print(f"Number of validation batches / 验证批次数: {len(val_loader)}")
print(f"Number of test batches / 测试批次数: {len(test_loader)}")

In [ ]:
# Training configuration / 训练配置
criterion = nn.MSELoss()  # Mean Squared Error loss / 均方误差损失
optimizer = Adam(model.parameters(), lr=0.001)  # Adam optimizer / Adam 优化器
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, 
                              min_lr=1e-7, verbose=True)

# Early stopping parameters / 早停参数
patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0
best_model_state = None

# Training loop / 训练循环
num_epochs = 100
history = {
    'train_loss': [],
    'val_loss': [],
    'train_mae': [],
    'val_mae': []
}

print("Training the RNN model... / 训练 RNN 模型...")
print("=" * 60)

for epoch in range(num_epochs):
    # Training phase / 训练阶段
    model.train()
    train_loss = 0.0
    train_mae = 0.0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_mae += torch.mean(torch.abs(outputs - batch_y)).item()
    
    train_loss /= len(train_loader)
    train_mae /= len(train_loader)
    
    # Validation phase / 验证阶段
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            val_loss += loss.item()
            val_mae += torch.mean(torch.abs(outputs - batch_y)).item()
    
    val_loss /= len(val_loader)
    val_mae /= len(val_loader)
    
    # Store history / 存储历史
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_mae'].append(train_mae)
    history['val_mae'].append(val_mae)
    
    # Learning rate scheduling / 学习率调度
    scheduler.step(val_loss)
    
    # Early stopping / 早停
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    
    # Print progress / 打印进度
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.6f}, Train MAE: {train_mae:.6f}")
        print(f"  Val Loss:   {val_loss:.6f}, Val MAE:   {val_mae:.6f}")
        print(f"  Best Val Loss: {best_val_loss:.6f}")
    
    # Check early stopping / 检查早停
    if epochs_no_improve >= patience:
        print(f"\nEarly stopping triggered after epoch {epoch+1}")
        print(f"提前停止在第 {epoch+1} 轮后触发")
        break

# Restore best model / 恢复最佳模型
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\nRestored best model with validation loss: {best_val_loss:.6f}")
    print(f"恢复验证损失为 {best_val_loss:.6f} 的最佳模型")

print("\nTraining completed! / 训练完成！")
print("=" * 60)

### 6.1 训练历史可视化 / Training History Visualization

绘制训练和验证损失曲线以检查过拟合

Plot training and validation loss curves to check for overfitting

In [ ]:
def plot_training_history(history):
    """
    Plot training and validation metrics
    绘制训练和验证指标
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Training Loss / 训练损失')
    axes[0].plot(history['val_loss'], label='Validation Loss / 验证损失')
    axes[0].set_title('Model Loss During Training / 训练期间的模型损失')
    axes[0].set_xlabel('Epoch / 轮次')
    axes[0].set_ylabel('Loss (MSE) / 损失 (MSE)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # MAE
    axes[1].plot(history['train_mae'], label='Training MAE / 训练 MAE')
    axes[1].plot(history['val_mae'], label='Validation MAE / 验证 MAE')
    axes[1].set_title('Model MAE During Training / 训练期间的模型 MAE')
    axes[1].set_xlabel('Epoch / 轮次')
    axes[1].set_ylabel('MAE')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

## 步骤 7: 模型评估 / Step 7: Model Evaluation

### 7.1 生成预测 / Make Predictions

在测试集上生成预测并反向转换到原始尺度

Generate predictions on test set and inverse transform to original scale

In [ ]:
# Make predictions / 生成预测
model.eval()

with torch.no_grad():
    # Training set predictions / 训练集预测
    y_train_pred = []
    for batch_X, _ in train_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        y_train_pred.append(outputs.cpu().numpy())
    y_train_pred = np.concatenate(y_train_pred)
    
    # Test set predictions / 测试集预测
    y_test_pred = []
    for batch_X, _ in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        y_test_pred.append(outputs.cpu().numpy())
    y_test_pred = np.concatenate(y_test_pred)

# Inverse transform to original scale / 反向转换到原始尺度
y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, 1)).flatten()
y_train_pred_inv = scaler_y.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()

y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_test_pred_inv = scaler_y.inverse_transform(y_test_pred.reshape(-1, 1)).flatten()

print("Predictions generated successfully! / 预测生成成功！")
print(f"\nSample predictions (first 5) / 预测样本（前5个）:")
print(f"Actual / 实际:    {y_test_actual[:5]}")
print(f"Predicted / 预测: {y_test_pred_inv[:5]}")

### 7.2 计算评估指标 / Calculate Evaluation Metrics

计算标准回归指标以评估模型性能

Compute standard regression metrics to assess model performance

In [ ]:
def calculate_metrics(y_true, y_pred, set_name='Test'):
    """
    Calculate and display regression metrics
    计算并显示回归指标
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{set_name} Set Performance Metrics / {set_name}集性能指标:")
    print("=" * 50)
    print(f"MSE (Mean Squared Error):        {mse:.4f}")
    print(f"RMSE (Root Mean Squared Error):  {rmse:.4f}")
    print(f"MAE (Mean Absolute Error):       {mae:.4f}")
    print(f"R² Score:                        {r2:.4f}")
    print(f"MAPE (Mean Absolute % Error):    {mape:.2f}%")
    print("=" * 50)
    
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

# Calculate metrics for both sets / 计算两个集合的指标
train_metrics = calculate_metrics(y_train_actual, y_train_pred_inv, 'Training / 训练')
test_metrics = calculate_metrics(y_test_actual, y_test_pred_inv, 'Test / 测试')

## 步骤 8: 结果可视化 / Step 8: Results Visualization

### 8.1 实际值 vs 预测值 / Actual vs Predicted Values

In [ ]:
# Plot predictions vs actual / 绘制预测 vs 实际
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Test set - full view / 测试集 - 完整视图
axes[0].plot(y_test_actual, label='Actual / 实际', alpha=0.7, linewidth=1.5)
axes[0].plot(y_test_pred_inv, label='Predicted / 预测', alpha=0.7, linewidth=1.5)
axes[0].set_title('Oil Temperature Prediction - Test Set (Full View) / 油温预测 - 测试集（完整视图）', fontsize=14)
axes[0].set_xlabel('Time Step / 时间步')
axes[0].set_ylabel('Oil Temperature (°C) / 油温 (°C)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test set - zoomed view (first 500 points) / 测试集 - 放大视图（前500个点）
zoom_range = 500
axes[1].plot(y_test_actual[:zoom_range], label='Actual / 实际', alpha=0.7, linewidth=1.5)
axes[1].plot(y_test_pred_inv[:zoom_range], label='Predicted / 预测', alpha=0.7, linewidth=1.5)
axes[1].set_title(f'Oil Temperature Prediction - Test Set (First {zoom_range} Points) / 油温预测 - 测试集（前{zoom_range}个点）', fontsize=14)
axes[1].set_xlabel('Time Step / 时间步')
axes[1].set_ylabel('Oil Temperature (°C) / 油温 (°C)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 8.2 散点图：预测值 vs 实际值 / Scatter Plot: Predicted vs Actual

In [ ]:
# Scatter plot / 散点图
plt.figure(figsize=(10, 8))
plt.scatter(y_test_actual, y_test_pred_inv, alpha=0.5, s=20)
plt.plot([y_test_actual.min(), y_test_actual.max()], 
         [y_test_actual.min(), y_test_actual.max()], 
         'r--', lw=2, label='Perfect Prediction / 完美预测')
plt.xlabel('Actual Oil Temperature (°C) / 实际油温 (°C)', fontsize=12)
plt.ylabel('Predicted Oil Temperature (°C) / 预测油温 (°C)', fontsize=12)
plt.title('Predicted vs Actual Oil Temperature (Test Set) / 预测 vs 实际油温（测试集）', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 8.3 残差分析 / Residual Analysis

In [ ]:
# Calculate residuals / 计算残差
residuals = y_test_actual - y_test_pred_inv

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Residual plot / 残差图
axes[0].scatter(y_test_pred_inv, residuals, alpha=0.5, s=20)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Oil Temperature (°C) / 预测油温 (°C)')
axes[0].set_ylabel('Residuals (°C) / 残差 (°C)')
axes[0].set_title('Residual Plot / 残差图')
axes[0].grid(True, alpha=0.3)

# Residual distribution / 残差分布
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals (°C) / 残差 (°C)')
axes[1].set_ylabel('Frequency / 频率')
axes[1].set_title('Distribution of Residuals / 残差分布')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Residual Statistics / 残差统计:")
print(f"Mean / 均值: {residuals.mean():.4f}°C")
print(f"Std / 标准差: {residuals.std():.4f}°C")
print(f"Min / 最小值: {residuals.min():.4f}°C")
print(f"Max / 最大值: {residuals.max():.4f}°C")

## 步骤 9: 结论和总结 / Step 9: Conclusion and Summary

### 模型性能总结 / Model Performance Summary

In [ ]:
# Create summary comparison / 创建总结对比
summary_df = pd.DataFrame({
    'Metric / 指标': ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE (%)'],
    'Training / 训练': [
        train_metrics['MSE'],
        train_metrics['RMSE'],
        train_metrics['MAE'],
        train_metrics['R2'],
        train_metrics['MAPE']
    ],
    'Test / 测试': [
        test_metrics['MSE'],
        test_metrics['RMSE'],
        test_metrics['MAE'],
        test_metrics['R2'],
        test_metrics['MAPE']
    ]
})

print("\n" + "=" * 60)
print("RNN MODEL PERFORMANCE SUMMARY / RNN 模型性能总结")
print("=" * 60)
print(summary_df.to_string(index=False))
print("=" * 60)

print("\n### Key Observations / 关键观察:")
print(f"1. The model achieves R² score of {test_metrics['R2']:.4f} on test set")
print(f"   模型在测试集上达到 R² 分数 {test_metrics['R2']:.4f}")
print(f"2. Average prediction error (MAE): {test_metrics['MAE']:.4f}°C")
print(f"   平均预测误差 (MAE): {test_metrics['MAE']:.4f}°C")
print(f"3. MAPE: {test_metrics['MAPE']:.2f}%")

if abs(train_metrics['R2'] - test_metrics['R2']) < 0.05:
    print("4. Model shows good generalization (minimal overfitting)")
    print("   模型显示良好的泛化能力（过拟合最小）")
else:
    print("4. Consider regularization or more data to reduce overfitting")
    print("   考虑正则化或更多数据以减少过拟合")

print("\n### Model Architecture / 模型架构:")
print(f"- Sequence Length / 序列长度: {seq_length} time steps (6 hours / 6小时)")
print(f"- RNN Units / RNN 单元: {hidden_sizes}")
print(f"- Total Parameters / 总参数数: {total_params:,}")

print("\n### Next Steps / 下一步:")
print("1. Compare with other models (Linear Regression, Random Forest, MLP)")
print("   与其他模型比较（线性回归、随机森林、MLP）")
print("2. Try advanced architectures (LSTM, GRU)")
print("   尝试先进架构（LSTM、GRU）")
print("3. Experiment with different sequence lengths")
print("   尝试不同的序列长度")
print("4. Feature engineering to improve performance")
print("   特征工程以提高性能")
print("5. Compare with SOTA model (Informer)")
print("   与 SOTA 模型（Informer）比较")

### 可选：保存模型 / Optional: Save the Model

In [ ]:
# Uncomment to save the model / 取消注释以保存模型
# import os
# os.makedirs('../models', exist_ok=True)
# torch.save(model.state_dict(), '../models/rnn_model.pth')
# print("Model saved to ../models/rnn_model.pth")
# print("模型已保存到 ../models/rnn_model.pth")